In [1]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from preprocessing import preprocess

In [3]:
%%time
df = pd.read_excel('indeed_results_new.xlsx')
for n, file in enumerate(os.listdir('data/')):
    if file.endswith('.xlsx'):
        print(f'{n+1}, {file}')
        df = df.append(pd.read_excel('data/'+file), ignore_index=True)
        df.drop_duplicates().to_excel('indeed_results_new.xlsx', index=False)
        os.remove('data/'+file)

Wall time: 1.27 s


In [4]:
df = pd.read_excel('indeed_results_new.xlsx')

In [5]:
df

,Title,Location,Company,Salary,Sponsored,Description,Time
0,PGT Art 'N' Craft,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹22,000 - ₹25,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-10
1,TGT BIOLOGY,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹22,000 - ₹28,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-22
2,TGT COMPUTER 6-8,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹16,000 - ₹20,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-22
3,Fashion Design faculty,"Chinchwad, Pune, Maharashtra",Times and Trends Academy Chinchwad,"₹20,000 a month",Sponsored,Job Summary\nIf you are interested in a high-g...,2020-02-17
4,Data Entry Back Office,"Gwalior, Madhya Pradesh",World Collection,"₹45,000 a month",Organic,"Locality- Birla Nagar, Kailash Nagar, Malanpur...",2020-02-25
...,...,...,...,...,...,...,...
17133,Accounts Executive,Pune,DVC Process Technologists,"₹15,000 - ₹20,000 a month",Organic,Job Summary\nAccounts Payable including bill b...,2020-03-04
17134,Club Lounge Agent,Pune,"Marriott International, Inc",NaN,Organic,"Posting Date Mar 02, 2020\nJob Number 20028713...",2020-03-04
17135,Customer Assistant,Pune,credence resource management,"₹20,668 - ₹35,800 a year",Organic,Greeting of the day\nwe are hiring urgently fo...,2020-02-25
17136,Inside Sales Executive,Pune,The Learners Camp,"₹4,00,000 a year",Organic,We sell high-quality online courses on a varie...,2020-02-28


In [6]:
df['Description'].unique()[np.random.randint(0, df['Description'].nunique())]

'If you have experience in native translations and language services, then we could be your partners!\nAt Games2win, we excel at making casual, snacky Mobile Games. All our games are global and played by consumers all over the world. This is why we need them to be translated into over 20 languages. At the moment, we do all our translation sourcing via Upwork. We want to understand if we can work directly with long term and reliable partners!\nWe are looking for Partners who:\n– Have native, local teams of translators in the countries and the languages we are targeting.\n– Be a professional business that can scale handle large volumes on our part.\nIf this sounds exciting, please fill-up the short form below and we will get in touch with you!\nhttps://forms.gle/9B8i8UfhmKeFYmodA'

In [9]:
def preprocessing_title_location_company(text):
    removal_notations = ["'", '"', '(', ')', '[', ']', '-', '/', '.', ',', '`', ':', '|', '\\', ';', '<', '>', '?', '~', '!', '#', '$', '%', '^', '*', '_', '+', '=', '{', '}']
    for removal_notation in removal_notations:
        for _ in range(20):
            text.replace(removal_notation, ' ')
    for _ in range(10):
        text = text.replace('&', 'and')
    for _ in range(10):
        text = text.replace("  ", ' ')
    return text

In [10]:
def preprocessing_description(text):
    removal_notations = ["'", '"', '(', ')', '[', ']', '-', '/', '.', ',', '`', ':', '|', '\\', ';', '<', '>', '?', '~', '!', '#', '$', '%', '^', '*', '_', '+', '=', '{', '}']
    for _ in range(100):
        text = text.replace('\n', ' ')
    for removal_notation in removal_notations:
        for _ in range(20):
            text.replace(removal_notation, ' ')
    for _ in range(10):
        text = text.replace('&', 'and')
    for _ in range(10):
        text = text.replace("  ", ' ')
    return text

In [11]:
def preprocessing_salary(text):
    removal_notations = ["'", '"', '(', ')', '[', ']', '-', '/', '.', '`', ':', '|', '\\', ';', '<', '>', '?', '~', '!', '#', '$', '%', '^', '*', '_', '+', '=', '{', '}']
    for removal_notation in removal_notations:
        for _ in range(20):
            text.replace(removal_notation, ' ')
    for _ in range(10):
        text = text.replace('&', 'and')
    for _ in range(10):
        text = text.replace(',', '')
    for _ in range(10):
        text = text.replace("  ", ' ')
    return text

In [12]:
df.dropna(inplace=True)

In [13]:
%%time
df['Title'] = df['Title'].apply(lambda x: preprocessing_title_location_company(x))
df['Location'] = df['Location'].apply(lambda x: preprocessing_title_location_company(x))
df['Company'] = df['Company'].apply(lambda x: preprocessing_title_location_company(x))
df['Description'] = df['Description'].apply(lambda x: preprocessing_description(x))
df['Salary'] = df['Salary'].apply(lambda x: preprocessing_salary(x))